In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(2)

In [2]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 100
cifar10_args.predict_batch_size = 100

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.2
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'SGD'
cifar10_args.scheduler = 'multistep'

activation = 'relu'
input_channel = 3
knpp = [32,64,96,128,160,192,224]

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, activation, False)),
    ('cnn2d', ((knpp[-1], knpp[-1], (2,2), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
#     ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3))),
    ('softmax', (1))
]


In [3]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()

Use GPU
Files already downloaded and verified
Files already downloaded and verified
add record: 02/19/2023 20:16
epoch: 0, train_loss: 2.1252, test_metric: 0.4107, time: 356.9508194923401
epoch: 1, train_loss: 2.0562, test_metric: 0.4476, time: 355.8755419254303
epoch: 2, train_loss: 2.0012, test_metric: 0.472, time: 355.8539776802063
epoch: 3, train_loss: 1.9513, test_metric: 0.5516, time: 356.31746554374695
epoch: 4, train_loss: 1.9117, test_metric: 0.567, time: 355.8401517868042
epoch: 5, train_loss: 1.8781, test_metric: 0.5706, time: 356.29543900489807
epoch: 6, train_loss: 1.8483, test_metric: 0.5297, time: 356.2676491737366
epoch: 7, train_loss: 1.8243, test_metric: 0.6599, time: 356.35733556747437
epoch: 8, train_loss: 1.803, test_metric: 0.6347, time: 356.7639925479889
epoch: 9, train_loss: 1.7813, test_metric: 0.607, time: 356.7088134288788
epoch: 10, train_loss: 1.7685, test_metric: 0.6829, time: 356.6649844646454
epoch: 11, train_loss: 1.7557, test_metric: 0.677, time: 356.8

epoch: 107, train_loss: 1.5421, test_metric: 0.8788, time: 355.9022147655487
epoch: 108, train_loss: 1.5434, test_metric: 0.8797, time: 355.8034317493439
epoch: 109, train_loss: 1.5428, test_metric: 0.8779, time: 355.71325612068176
epoch: 110, train_loss: 1.5423, test_metric: 0.8776, time: 355.72936034202576
epoch: 111, train_loss: 1.5422, test_metric: 0.8662, time: 355.773024559021
epoch: 112, train_loss: 1.5429, test_metric: 0.8819, time: 355.6285824775696
epoch: 113, train_loss: 1.5408, test_metric: 0.8827, time: 355.7171905040741
epoch: 114, train_loss: 1.541, test_metric: 0.8876, time: 355.81955456733704
epoch: 115, train_loss: 1.5445, test_metric: 0.8815, time: 355.6934995651245
epoch: 116, train_loss: 1.5419, test_metric: 0.879, time: 355.45053791999817
epoch: 117, train_loss: 1.5428, test_metric: 0.8764, time: 356.05016708374023
epoch: 118, train_loss: 1.5419, test_metric: 0.8848, time: 355.77485489845276
epoch: 119, train_loss: 1.5423, test_metric: 0.8825, time: 355.9451150894

epoch: 214, train_loss: 1.4778, test_metric: 0.9105, time: 357.6167347431183
epoch: 215, train_loss: 1.4781, test_metric: 0.9139, time: 357.56252908706665
epoch: 216, train_loss: 1.4777, test_metric: 0.9135, time: 357.6715018749237
epoch: 217, train_loss: 1.4776, test_metric: 0.9126, time: 357.4286172389984
epoch: 218, train_loss: 1.4787, test_metric: 0.9147, time: 357.51513409614563
epoch: 219, train_loss: 1.4771, test_metric: 0.9126, time: 357.4571716785431
epoch: 220, train_loss: 1.4772, test_metric: 0.9153, time: 357.4000895023346
epoch: 221, train_loss: 1.4774, test_metric: 0.9128, time: 357.50747418403625
epoch: 222, train_loss: 1.4772, test_metric: 0.9125, time: 357.5004892349243
epoch: 223, train_loss: 1.4774, test_metric: 0.9125, time: 357.90645360946655
epoch: 224, train_loss: 1.4769, test_metric: 0.9113, time: 357.4645664691925
epoch: 225, train_loss: 1.4763, test_metric: 0.9128, time: 499.4730951786041
epoch: 226, train_loss: 1.4769, test_metric: 0.9132, time: 612.384330511

In [4]:
pytorch_total_params = sum(p.numel() for p in driver.model.parameters() if p.requires_grad)
pytorch_total_params

1286250